<a href="https://colab.research.google.com/github/erazocar/classifiers_ml_hw/blob/main/kkk_classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HW5: Classifiers**
## **Required Libraries**

In [34]:
#Scientific computing libraries
import numpy as np
import os
import pandas as pd
import scipy as sp

#performance control
import time

#Libraries from Sklearn
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import LeaveOneOut
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


## **Function Definition**

In [ ]:
#Reader for each file, appended to a list.
def reader1(path):
  data = []
  filenames = os.listdir(path)
  for file in filenames:
    filepath = path + '/' + file
    d = pd.read_csv(filepath, header = None, sep=',')
    data.append(d)
  return data

In [6]:
#Reader for each file, appended to a list.
def reader2(path):
  data = []
  filenames = os.listdir(path)
  for file in filenames:
    filepath = path + '/' + file
    d = pd.read_csv(filepath, header = None, sep=' ')
    data.append(d)
  return data

In [ ]:
#KNN Classifier with num neighbors added
def training_KNN(train_data, train_label, test_data, neigh):
  start = time.time()
  classifier = KNN(n_neighbors=neigh)
  classifier.fit(train_data, np.ravel(train_label))
  end = time.time()
  pred = classifier.predict(test_data)

  #training model

  print("Time duration for KNN " + str(end - start) + "s.")
  return pred, classifier

In [24]:
#Train Naive Bayes estimator
def training_NB(x_train, y_train, x_test ,y_test):
  NB = MultinomialNB(alpha=1)
  start = time.time()
  result = NB.fit(x_train, np.ravel(y_train))
  pred = NB.predict(x_test)
  end = time.time()

  print("Accuracy: "+ str(NB.score(x_test, np.ravel(y_test))) + ", Time duration for NB " + str(end - start))
  return pred

In [3]:
#Create feature training set for Naive Bayes Estimation
def feature_conversion(training_set, word):
  #Get features of documents and words
  doc,doc_count= np.unique(training_set[0], return_counts=True)
  #Accumulate the count of documents per row and assign to list
  doc_count_sum = np.cumsum(doc_count).tolist()
  #Append the first line with 0s on the sum docs
  doc_count_sum.insert(0,0)

  X_traina = []

  for i, value1 in enumerate(doc_count_sum):
    try:
      val = doc_count_sum[i]
      end = doc_count_sum[i+1]
      rows = [0] * len(word)

      selec = training_set[1][val:end]
      ans = training_set[2][val:end]
      index = np.where(np.in1d(word, selec))[0].tolist()

      for (ind, replacement) in zip(index, ans):
        rows[ind] = replacement
      X_traina.append(rows)
    except IndexError:
      pass

  
  return sp.sparse.csr_matrix(X_traina)

## **Preprocessing**

In [ ]:
#Read paths and create contents for the documents.
#Please put the local paths were the queries and docs are located when running.
#Data for problem 1
folder1 = '/content/drive/MyDrive/MachineLearning/homework-5/heart'

example1 = reader1(folder1)

In [7]:
#Read paths and create contents for the documents.
#Please put the local paths were the queries and docs are located when running.
#Data for problem 1
folder2 = '/content/drive/MyDrive/MachineLearning/homework-5/20newsgroup'

example2 = reader2(folder2)

In [ ]:
#Read paths and create contents for the documents.
#Please put the local paths were the queries and docs are located when running.
#Data for problem 1
folder3 = '/content/drive/MyDrive/MachineLearning/homework-5/gisette'

example3 = reader2(folder3)

## **Problem 1**

Using Leave-one-out cross validation for different values of k

In [ ]:
#Evaluate which is the best accuracy for the cross validation
loocv = LeaveOneOut()

#List with k's iterating from 1 to 10
ks = range(1,11)

for i in ks:
  start1 = time.time()
  mod = KNN(n_neighbors=i)
  results_loocv = cross_val_score(mod, example1[0], np.ravel(example1[1]), cv = loocv, scoring='accuracy')
  end1 = time.time()
  print("Accuracy : %.2f%%" % (results_loocv.mean()*100) + f" for k={i}")
  print("Averaged error : %.2f%%" % ((1-results_loocv.mean())*100), end = '\n')
  print("Time duration for problem1 " + str(end1 - start1) + f"s for k={i}")
  print("")

Accuracy : 77.37% for k=1
Averaged error : 22.63%
Time duration for problem1 0.8773796558380127s for k=1

Accuracy : 77.37% for k=2
Averaged error : 22.63%
Time duration for problem1 0.8300397396087646s for k=2

Accuracy : 82.72% for k=3
Averaged error : 17.28%
Time duration for problem1 0.8476603031158447s for k=3

Accuracy : 81.48% for k=4
Averaged error : 18.52%
Time duration for problem1 0.8179936408996582s for k=4

Accuracy : 81.89% for k=5
Averaged error : 18.11%
Time duration for problem1 0.7978365421295166s for k=5

Accuracy : 83.54% for k=6
Averaged error : 16.46%
Time duration for problem1 0.8312366008758545s for k=6

Accuracy : 83.13% for k=7
Averaged error : 16.87%
Time duration for problem1 0.8271458148956299s for k=7

Accuracy : 81.89% for k=8
Averaged error : 18.11%
Time duration for problem1 0.8243300914764404s for k=8

Accuracy : 81.89% for k=9
Averaged error : 18.11%
Time duration for problem1 0.8216161727905273s for k=9

Accuracy : 82.30% for k=10
Averaged error : 17

In [ ]:
#Train model on the best k and reporting the labels
knn_values, knn_model = training_KNN(example1[0], example1[1], example1[2], 6)

Time duration for KNN 0.0019381046295166016s.


In [ ]:
#Predicted labers
knn_values

array([-1, -1,  1, -1,  1, -1, -1,  1, -1,  1,  1,  1, -1, -1, -1,  1,  1,
       -1, -1, -1,  1,  1,  1,  1, -1, -1, -1])

## **Problem 2**

In [8]:
#"Bag of words" for training the data
words = np.unique(example2[0][1])

In [9]:
#Total will take around 4 min of execution
X_train2 = feature_conversion(example2[0], words)
X_test2 = feature_conversion(example2[3], words)

In [25]:
#Prediction
NB_cl = training_NB(X_train2, example2[1], X_test2, example2[5])

Accuracy: 0.7846768820786143, Time duration for NB 0.11965441703796387


In [31]:
#Classifier and labeler
classif = classification_report(example2[5], NB_cl, target_names=example2[2][0])
print(classif)

                          precision    recall  f1-score   support

             alt.atheism       0.70      0.75      0.73       318
           comp.graphics       0.67      0.76      0.71       389
 comp.os.ms-windows.misc       0.82      0.53      0.65       391
comp.sys.ibm.pc.hardware       0.60      0.77      0.68       392
   comp.sys.mac.hardware       0.79      0.72      0.75       383
          comp.windows.x       0.82      0.78      0.80       390
            misc.forsale       0.91      0.62      0.73       382
               rec.autos       0.79      0.90      0.84       395
         rec.motorcycles       0.94      0.89      0.91       397
      rec.sport.baseball       0.96      0.88      0.92       397
        rec.sport.hockey       0.94      0.96      0.95       399
               sci.crypt       0.75      0.91      0.82       395
         sci.electronics       0.78      0.66      0.72       393
                 sci.med       0.89      0.82      0.85       393
         

In [33]:
#Print out in directory, please change the name of directory if neccesary.
savpath = "/content/drive/MyDrive/MachineLearning/homework-5"
name = "P2_Result.txt"

completeName = os.path.join(savpath, name)
file1 = open(completeName, "w")
file1.write(classif)
file1.close()

## **Problem 3**

Inside the list containing the files, index 0 and 2 correspond to the testing dataset (label and data, respectively), while index 1 and 3 correspond to the training dataset. (label and data respectively).

### **Part 1: KNN using given data**

Using a holdout validation approach, the given testing and training sets were evaluated to get the performance of the models. To obtain the best k, the same approach as for the first problem was used testing the k's on the models iteratively.

In [ ]:
#Using the same k leave one out cross validation for selecting the best k
ks2 = range(1,11)

for i in ks2:
  start2 = time.time()
  mod2 = KNN(n_neighbors=i)
  mod2.fit(example3[3], np.ravel(example3[1]))
  results = mod2.score(example3[2], np.ravel(example3[0]))
  end2 = time.time()
  print("Accuracy : %.2f%%" % (results*100) + f" for k={i}")
  print("Averaged error : %.2f%%" % ((1-results)*100), end = '\n')
  print("Time duration for problem 3 with original data " + str(end2 - start2) + f"s for k={i}")
  print("")

Accuracy : 96.20% for k=1
Averaged error : 3.80%

Accuracy : 96.10% for k=2
Averaged error : 3.90%

Accuracy : 96.60% for k=3
Averaged error : 3.40%

Accuracy : 96.40% for k=4
Averaged error : 3.60%

Accuracy : 96.10% for k=5
Averaged error : 3.90%

Accuracy : 96.50% for k=6
Averaged error : 3.50%

Accuracy : 96.20% for k=7
Averaged error : 3.80%

Accuracy : 96.50% for k=8
Averaged error : 3.50%

Accuracy : 96.30% for k=9
Averaged error : 3.70%

Accuracy : 96.40% for k=10
Averaged error : 3.60%



### **Part 2: KNN using PCA**

After trying with several component decomposition, the optimum k was found at number of components = 11 with k=6, grabbing from a list of k's from 0 to 10.

In [ ]:
#Using the same k leave one out cross validation for selecting the best k in PCA
ks3 = range(1,11)

for i in ks3:
  start3 = time.time()
  mod3 = KNN(n_neighbors=i)
  #create pipeline and feed the PCA decomposition function.
  pca = make_pipeline(StandardScaler(), PCA(n_components=11))
  pca.fit_transform(example3[3], np.ravel(example3[1]))
  mod3.fit(pca.transform(example3[3]), np.ravel(example3[1]))
  results2 = mod3.score(pca.transform(example3[2]), np.ravel(example3[0]))
  end3 = time.time()
  print("Accuracy : %.2f%%" % (results2*100) + f" for k={i}")
  print("Averaged error : %.2f%%" % ((1-results2)*100), end = '\n')
  print("Time duration for problem 3 with PCA " + str(end3 - start3) + f"s for k={i}")
  print("")

Accuracy : 96.20% for k=1
Averaged error : 3.80%
Time duration for problem 3 with PCA 4.572713613510132s for k=1

Accuracy : 95.70% for k=2
Averaged error : 4.30%
Time duration for problem 3 with PCA 4.391347408294678s for k=2

Accuracy : 96.60% for k=3
Averaged error : 3.40%
Time duration for problem 3 with PCA 4.420243740081787s for k=3

Accuracy : 96.30% for k=4
Averaged error : 3.70%
Time duration for problem 3 with PCA 4.329538822174072s for k=4

Accuracy : 96.70% for k=5
Averaged error : 3.30%
Time duration for problem 3 with PCA 4.423214912414551s for k=5

Accuracy : 97.00% for k=6
Averaged error : 3.00%
Time duration for problem 3 with PCA 4.496803283691406s for k=6

Accuracy : 96.50% for k=7
Averaged error : 3.50%
Time duration for problem 3 with PCA 4.522700786590576s for k=7

Accuracy : 96.30% for k=8
Averaged error : 3.70%
Time duration for problem 3 with PCA 4.455880403518677s for k=8

Accuracy : 96.20% for k=9
Averaged error : 3.80%
Time duration for problem 3 with PCA 4.